测试 WorkFlow.InitStates 中的 linear_comb_of_scar_states() 函数

In [1]:
import torch as tc
import numpy as np

from WorkFlow.InitStates import xorX_state_m

In [2]:
state = xorX_state_m(1, 6, device=tc.device('cpu'), dtype=tc.complex64)
print(state.shape)

torch.Size([1, 2, 2, 2, 2, 2, 2])


In [3]:
def linear_comb_of_scar_states(number:int, length:int, device=tc.device('cuda:0'), dtype=tc.complex128, **kwargs):
    coefficients = np.random.random(size=[number, length//2]) + 1.j * np.random.random(size=[number, length//2])
    norm = np.sum(coefficients * coefficients.conj(), axis=1, keepdims=True).real
    coefficients = coefficients / np.sqrt(norm)
    coefficients = tc.from_numpy(coefficients).to(device=device, dtype=dtype)
    states = tc.zeros(size=[number]+[2]*length, device=device, dtype=dtype)
    for i in range(length // 2):
        states = states + tc.einsum('ij, jk...l->ik...l', coefficients[:, i].unsqueeze(1), xorX_state_m(i, length, device, dtype))
        pass
    return states

In [4]:
states = linear_comb_of_scar_states(number=1, length=10, device=tc.device('cpu'), dtype=tc.complex64)

In [5]:
print(states.shape)

torch.Size([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [7]:
states = states.reshape([1, -1])
norm = tc.einsum('ni, ni->n', states, states.conj())
print(norm)

tensor([1.0000+2.7061e-09j])
